In [ ]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import time
from selenium import webdriver

In [ ]:
#URL to be scraped
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

In [ ]:
#Review html format
#print(soup.prettify())

In [ ]:
#Narrow the div and class to see how best to loop through
results = soup.find_all('div', class_="content_title")
results

In [ ]:
# A blank list to hold the headlines
news_titles = []
# Loop over div elements
for result in results:
    # Identify the anchor...
    if (result.a):
        # And the anchor has non-blank text...
        if (result.a.text):
            # Append thext to the list
            news_titles.append(result)
news_titles

In [ ]:
finalnewstitles = []
# Print only the headlines
for x in range(6):
    var=news_titles[x].text
    newvar = var.strip('\n\n')
    finalnewstitles.append(newvar)
finalnewstitles

In [ ]:
#Find classification for description paragraph below title
presults = soup.find_all('div', class_="rollover_description_inner")
presults

In [ ]:
news_p = []
# Loop through the div results to pull out just the text
for x in range(6):
    var=presults[x].text
    newvar = var.strip('\n\n')
    news_p.append(newvar)
news_p

### JPL Mars Space Images - Featured Image

In [ ]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
#executable_path = {'executable_path': 'chromedriver.exe'}
#browser = Browser('chrome', **executable_path, headless=False)
#browser.visit(url)
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [ ]:
#Review html format
print(soup.prettify())

In [ ]:
imgs = soup.find_all('a', class_="fancybox")
imgs

In [ ]:
# pull image link
feat_img = []
for img in imgs:
    pic = img['data-fancybox-href']
    feat_img.append(pic)

featured_image_url = 'https://www.jpl.nasa.gov' + pic
featured_image_url

### Mars Weather

Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

In [ ]:
# Setting up windows browser with chromedriver

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url='https://twitter.com/marswxreport?lang=en'

browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

### Mars Facts

In [ ]:
# Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing 
# facts about the planet including Diameter, Mass, etc.

#Use Pandas to convert the data to a HTML table string.

In [ ]:
url = 'https://space-facts.com/mars/'
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [ ]:
#Review html format
#print(soup.prettify())

In [ ]:
tables = pd.read_html(url)
tables[0]

In [ ]:
marsdf = tables[0]
marsdf

In [ ]:
marsdf.columns = ['Stat', 'Measurement']
marsdf.head()

In [ ]:
s = pd.Series(marsdf['Stat'])
marsdf['Stat'] = s.str.strip(':')
marsdf

In [ ]:
marsdf = marsdf.set_index('Stat')
marsdf

In [ ]:
#Use to_html method to generate HTML tables from df
html_table = marsdf.to_html()
html_table

In [ ]:
#Save as html file
marsdf.to_html('mars_table.html')

### Mars Hemispheres

In [ ]:
# Visit the USGS Astrogeology site [here]
# (https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution 
# images for each of Mar's hemispheres.

# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the 
# hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

# Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one 
# dictionary for each hemisphere.

# Example:
#hemisphere_image_urls = [
#    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#    {"title": "Cerberus Hemisphere", "img_url": "..."},
#    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
#]

In [ ]:
#URL to be scraped
url = 'https://www.usgs.gov/centers/astrogeology-science-center'
response = requests.get(url)

In [ ]:
# Setting up windows browser with chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# Setting url for alternate browser
url = 'https://www.usgs.gov/centers/astrogeology-science-center'
browser.visit(url)

In [ ]:
nextpage_urls = []
imgtitles = []
base_url = 'https://astrogeology.usgs.gov'

# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Retrieve all elements that contain hemisphere photo info
divs = soup.find_all('div', class_='description')

# Iterate through each div to pull titles and make list of hrefs to iterate through
counter = 0
for div in divs:
        # Use Beautiful Soup's find() method to navigate and retrieve attributes
    link = div.find('a')
    href=link['href']
    img_title = div.a.find('h3')
    img_title = img_title.text
    imgtitles.append(img_title)
    next_page = base_url + href
    nextpage_urls.append(next_page)
    counter = counter+1
    if (counter == 4):
        break
print(nextpage_urls)
print(imgtitles)

In [ ]:
# Creating loop for high resolution photo on next page

my_images=[]
for nextpage_url in nextpage_urls:
    url = nextpage_url
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    link2 = soup.find('img', class_="wide-image")
    forfinal = link2['src']
    full_img = base_url + forfinal
    my_images.append(full_img)
    nextpage_urls = []
my_images

In [ ]:
# Creating final list of dictionaries
# values - imgtitles and my_images
#ckeys- img_url and title
hemisphere_image_urls = []

cerberus = {'title':imgtitles[0], 'img_url': my_images[0]}
schiaparelli = {'title':imgtitles[1], 'img_url': my_images[1]}
syrtis = {'title':imgtitles[2], 'img_url': my_images[2]}
valles = {'title':imgtitles[3], 'img_url': my_images[3]}

hemisphere_image_urls = [cerberus, schiaparelli, syrtis, valles]
print(hemisphere_image_urls)